The task

1) File: http://www.almhuette-raith.at/apache-log/access.log

2) Use PySpark to processing this log

3) Reduce the set to only records with the 404 response code

4) Get the request field like “GET /icons/text.gif HTTP/1.1”

5) Get the URI from the request “/icons/text.gif”

6) Get top 10 unique URIs. 

7) Send me your program with the result file

In [18]:
import pyspark
import wget
import re 
from operator import add
from operator import itemgetter
import pandas as pd

In [2]:
sc = pyspark.SparkContext('local[*]')

In [3]:
# just to check it is working 
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)


[601, 634, 585, 111, 680]

In [5]:
logs = wget.download('http://www.almhuette-raith.at/apache-log/access.log')
#logs = 'access.log'

In [6]:
file_in = sc.textFile(logs)
print('number of lines in file: %s' % file_in.count())
chars = file_in.map(lambda s: len(s)).reduce(add)
print('number of characters in file: %s' % chars)


number of lines in file: 1485704
number of characters in file: 298798468


In [16]:
input = sc.textFile(logs)
# resular expression for the 404 and GET /icons/text.gif HTTP/1.1 conditions
# examle of log
#83.167.113.100 - - [12/Dec/2015:18:31:25 +0100] "POST /administrator/index.php HTTP/1.1" 200 4494 "http://almhuette-raith.at/administrator/" "Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20100101 Firefox/34.0" "-"
#109.169.248.247 - - [12/Dec/2015:18:25:11 +0100] "GET /administrator/ HTTP/1.1" 200 4263 "-" "Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20100101 Firefox/34.0" "-"
reg = re.compile(r'^.* - - \[.*\] "GET \/icons\/text\.gif HTTP\/1\.1" 404 [0-9-]+ "(.*)" ".*" ".*"$')

top_uri = input \
    .map(lambda line: re.match(reg, line)) \
    .filter(lambda match: match != None) \
    .map(lambda match: match.groups()[0]) \
    # removing '-' from the dictionary
    .filter(lambda match: match.split()[0] != '-') \
    .map(lambda request: request.split()[0]) \
    .countByValue()
    

In [17]:
top_uri

defaultdict(int,
            {'http://almhuette-raith.at/apache-log/': 26,
             'http://www.almhuette-raith.at/apache-log/': 310,
             'http://www.almhuette-raith.at/apache-log/?D=A': 4,
             'http://www.almhuette-raith.at/apache-log/?D=D': 2,
             'http://www.almhuette-raith.at/apache-log/?M=A': 3,
             'http://www.almhuette-raith.at/apache-log/?N=A': 1,
             'http://www.almhuette-raith.at/apache-log/?N=D': 1,
             'http://www.almhuette-raith.at/apache-log/?S=A': 1})

In [24]:
# saving result to pandas DataFrame
df = pd.DataFrame.from_dict(top_uri, orient = 'index')

In [26]:
#saving to csv file
df.to_csv('result.csv')